# 1. Rekognition Re-Index Solution

This notebook contains scripts to help you deploy and use the solution.

⚠️ **Please use notebook 0-Data-Preparation before running this notebook** ⚠️ 

**Notebook Steps:**
1. Import libraries and clients
2. Define variables
3. Deploy and launch the solution
4. Analyze the results

## 1. Import Libraries and Clients

In [ ]:
import boto3, json, os, logging, sagemaker, time
from botocore.exceptions import ClientError
from zipfile import ZipFile
rekClient = boto3.client('rekognition')
dynamoClient = boto3.client('dynamodb')
sfClient = boto3.client('stepfunctions')
cfClient = boto3.client('cloudformation')
sm_session = sagemaker.Session()
boto3_session = boto3.session.Session()
boto3_region = boto3_session.region_name

## 2. Define Variables

If you have run the previous notebook you will find the details in the last cell.

In [ ]:
old_CollectionId =""
new_CollectionId =""
s3_bucket_name =""
records_folder =""
records_filename =""

In [ ]:
stack_name = "" ## Specify a unique name for the stack
RekognitionIndexFacesQualityFilter = "AUTO"
RekognitionIndexFacesTPSLimit = "50"

## 3. Deploy and launch the solution

#### 3.1 Prepare and upload the necessary assets 

In [ ]:
lambda_folder = "assets/lambda_functions"
for file in os.listdir(lambda_folder):
    root,ext = os.path.splitext(file)
    if ext == ".py":
        with ZipFile("{}/{}.zip".format(lambda_folder,root),'w') as zip:
            zip.write("{}/{}".format(lambda_folder,file),file)
            zip.close()
        asset_s3_uri = sm_session.upload_data("{}/{}.zip".format(lambda_folder,root), s3_bucket_name, lambda_folder)
        print("Your function asset is located in {}".format(asset_s3_uri))

#### 3.2 Deploy the Amazon CloudFormation template 

The cloudformation stack deployment should take around 3 minutes.

In [ ]:
with open('{}/{}'.format("assets/cloudformation_template","template.yaml"), 'r') as file:
    template_body = file.read()
    file.close()
    
response_cft = cfClient.create_stack(
    StackName= stack_name,
    TemplateBody= template_body,
    Parameters=[
        {
            'ParameterKey': 'AssetsBucket',
            'ParameterValue': s3_bucket_name,
        },
        {
            'ParameterKey': 'RekognitionIndexFacesQualityFilter',
            'ParameterValue': RekognitionIndexFacesQualityFilter,
        },
        {
            'ParameterKey': 'RekognitionIndexFacesTPSLimit',
            'ParameterValue': RekognitionIndexFacesTPSLimit,
        },
    ],
    Capabilities=['CAPABILITY_IAM','CAPABILITY_NAMED_IAM','CAPABILITY_AUTO_EXPAND'],
    TimeoutInMinutes=20,
    OnFailure='ROLLBACK'
)

In [ ]:
def get_status(stackId):
    cft_status = cfClient.describe_stacks(
        StackName=stackId
    )
    return cft_status["Stacks"][0]

In [ ]:
print("Creating Stack...")
while get_status(response_cft["StackId"])["StackStatus"] == "CREATE_IN_PROGRESS":
    print(".", end ="")
    time.sleep(10)
print("Stack finished launching with state:", get_status(response_cft["StackId"])["StackStatus"])
sf_arn = get_status(response_cft["StackId"])["Outputs"][0]["OutputValue"]
print("Step Function Workflow ARN:", sf_arn)

#### 3.3 Launch a new Step Functions Workflow to ReIndex the collection

In [ ]:
execution_response = sfClient.start_execution(
    stateMachineArn=sf_arn,
    input=json.dumps({"bucket": s3_bucket_name,"key": "{}/{}".format(records_folder,records_filename)})
)
execution_arn = execution_response["executionArn"]

In [ ]:
def get_execution_status(execution_arn):
    execution_status = sfClient.describe_execution(
        executionArn=execution_arn
    )
    return execution_status["status"]

In [ ]:
print("Reindexing face collection...")
while get_execution_status(execution_arn) not in ["SUCCEEDED","FAILED"]:
    print(".", end ="")
    time.sleep(10)
print("Execution finished launching with state:", get_execution_status(execution_arn))

## 4. Analyze the results

In [ ]:
def list_collection_faces(collection_id):
    response = rekClient.list_faces(
        CollectionId=collection_id
    )
    print("Number Faces in {} is : {} ".format(collection_id, len(response["Faces"])))  

In [ ]:
list_collection_faces(old_CollectionId)
list_collection_faces(new_CollectionId)

#### Analyze Results

In [ ]:
response_scan = dynamoClient.scan(
        TableName=get_status(response_cft["StackId"])["Outputs"][1]["OutputValue"]
)

In [ ]:
new_faces_found = 0
for face in response_scan["Items"]:
    if (face["IsNewFace"]["S"]) == True:
        new_faces_found = new_faces_found +1

In [ ]:
print("Total faces reindexed:",response_scan["Count"])
print("New faces found:",new_faces_found)